# Hypergraphs
[Run notebook in Google Colab](https://colab.research.google.com/github/pathpy/pathpy/blob/master/doc/tutorial/hypergraphs.ipynb)  
[Download notebook](https://github.com/pathpy/pathpy/raw/master/doc/tutorial/hypergraphs.ipynb)

The `pathpy` package provides special support for the analysis of hypergraphs data via its `HyperGraph` class. It is suitable for data that captures unordered relations ${u,v,w}$.

To get started with `pathpy` we first import `pathpy` and assign the local alias `pp`:

In [ ]:
pip install git+git://github.com/pathpy/pathpy.git

In [1]:
import pathpy as pp
from pathpy.core.hyperedge import HyperEdge
from pathpy.models.hypergraph import HyperGraph

## Creating hypergraphs

For this purpose `pathpy` provides the `HyperGraph` class. Calling the constructor will return an instance that represents an empty hypergraph with no nodes and no hyperedges. 

Printing the `HyperGraph` object will give a short string summary which tells whether the hypergraph allows multi-edges, as well as the number of unique nodes and links.

In [2]:
hg = HyperGraph()
print(hg)

Uid:			0x7fe3a58025b0
Type:			HyperGraph
Multi-Edges:		False
Number of nodes:	0
Number of edges:	0


The simplest way to add nodes and edges is to call the functions `add_node` and `add_edge`. In both cases, we can simply pass unique string identifiers of nodes, which will then be used as UIDs of the underlying node objects. To create hypergraph with three nodes and two edges, we can write: 

In [3]:
hg = HyperGraph(multiedges=True,uid='ExampleNetwork')
hg.add_node('a')
hg.add_node('b')
hg.add_node('c')
hg.add_edge('a', 'b')
hg.add_edge('a','b', 'c')
print(hg)

Uid:			ExampleNetwork
Type:			HyperGraph
Multi-Edges:		True
Number of nodes:	3
Number of edges:	2


Unless we want to explicitly add isolated nodes with no incident edges, we can omit the explicit call of the `add_node` function. If we add hyperedges any node that does not exist already will be created and added automatically. If we want to check explicitly whether a node exists before creating and edge, we can test this with the `in` operator on the set of node UIDS available via `HyperGraph.nodes.uids`:

In [4]:
print('d' in hg.nodes.uids)

False


The following code will automatically add a new node `d`, along with a new hyperedge {`a`,`c`,`d`}.

In [5]:
hg.add_edge('a','c','d')
print(hg)

Uid:			ExampleNetwork
Type:			HyperGraph
Multi-Edges:		True
Number of nodes:	4
Number of edges:	3


In [6]:
print('d' in hg.nodes.uids)

True


To count the number of nodes and hyperedges in a network we can use the `number_of_nodes` and `number_of_edges` functions, or we could can compute `len` of `HyperGraph.nodes` and `HyperGraph.edges`:

In [7]:
print('HyperGraph has {0} nodes and {1} edges'.format(hg.number_of_nodes(), hg.number_of_edges()))
print('Number of nodes: {0}'.format(len(hg.nodes)))
print('Number of edges: {0}'.format(len(hg.edges)))

HyperGraph has 4 nodes and 3 edges
Number of nodes: 4
Number of edges: 3


### Node and Edge objects

In the simple example above, we generated nodes and edges by calling the `add_node` and `add_edge` function of the network instance. Internally, nodes and edges are represented as objects of type `Node` and `HyperEdge` that can be referenced by one or more instances of type `HyperGraph`. Just like a `HyperGraph`, each instance of a `Node` and `HyperEdge` has a UID. In the example above, `pathpy` has automatically created `Node` and `HyperEdge` instances and has assigned the UIDs `a`, `b`, `c`, and `d` to those nodes. We can access those node objects via the node container `HyperGraph.nodes`. We can iterate through this dictionary to print a summary of all node objects referenced with a hypergraph object:

In [8]:
for v in hg.nodes:
    print(v)

Uid:		a
Type:		Node

Uid:		b
Type:		Node

Uid:		c
Type:		Node

Uid:		d
Type:		Node



We can also use the uid of a node to access a specific node object in a network by using the uid as an index to the `nodes` container:

In [10]:
print(hg.nodes['a'])

Uid:		a
Type:		Node



Similar to `nodes`, the `edges` container of the hypergraph contains all hyperedges of a network and each hyperedge is actually stored as an `HyperEdge` object. Let us iterate through the edges container of network `hg` to better understand this:

In [11]:
for e in hg.edges:
    print('---')
    print(e)

---
Uid:		0x7fe3a5802700
Type:		HyperEdge
Nodes:		{'b', 'a'}

---
Uid:		0x7fe3a5802d90
Type:		HyperEdge
Nodes:		{'c', 'b', 'a'}

---
Uid:		0x7fe3a5802fd0
Type:		HyperEdge
Nodes:		{'c', 'a', 'd'}



We see that the edge container contains one `HyperEdge` object instance for each hyperedge that we added before. Each `HyperEdge` has again a unique identifier, which has been automatically created in our example above. Just like for `Node` or `HyperGraph` objects, we can manually create a hyperedge object with a custom UID that connects the nodes `a`, `b` and `c` as follows:

In [12]:
edge = HyperEdge('a','b','c', uid='MyHyperEdge')
print(edge)

Uid:		MyHyperEdge
Type:		HyperEdge
Nodes:		{'c', 'b', 'a'}



This `HyperEdge` object has a different UID than the existing edge between nodes `a`, `b` and `c`, which is why we can add it to network `hg` even though this network already contains an edge (with a different UID) between nodes `a`, `b` and `c`:

In [13]:
hg.add_edge(edge)
print(hg)

Uid:			ExampleNetwork
Type:			HyperGraph
Multi-Edges:		True
Number of nodes:	4
Number of edges:	4


The summary of the hypergraph confirms that the network now contains four hyperedges. This native support for multi-edge networks is an important feature of `pathpy`. It also means that every pair of nodes can be connected by more than one edge. We can access those edges via the `HyperGraph.edges` container in multiple ways. First, we can simply iterate through the edge objects as shown before. Second, we can directly access an `HyperEdge` with a given UID as follows:

In [14]:
print(hg.edges['MyHyperEdge'])

Uid:		MyHyperEdge
Type:		HyperEdge
Nodes:		{'c', 'b', 'a'}



Finally, we often want to access those hyperedges that connect a specific set of nodes. We can thus alternatively pass the node uids as index to `HyperGraph.edges`. Since multiple edges between the same pair of nodes are possible, this generally returns a list of HyperEdge objects, which - in the case of the node pair `a` and `b` - contains two different edge objects with different UIDs.

In [15]:
print(hg.edges['a','b','c'])

{HyperEdge {'c', 'b', 'a'}, HyperEdge MyHyperEdge}


Since the relationships between nodes inside a `HyperEdge` are unorderd, we can access a `HyperEdge` with any valid combination of nodes:

In [16]:
print(hg.edges['a','c','b'])

{HyperEdge {'c', 'b', 'a'}, HyperEdge MyHyperEdge}


In [17]:
print(hg.edges['b','a','c'])

{HyperEdge {'c', 'b', 'a'}, HyperEdge MyHyperEdge}


In [18]:
print(hg.edges['c','b','a'])

{HyperEdge {'c', 'b', 'a'}, HyperEdge MyHyperEdge}
